In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
labels = ['happy', 'neutral', 'sad']
img_size = 64
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in tqdm(os.listdir(path)):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
images_data = get_data("gdrive/My Drive/emotions_data")

100%|██████████| 6078/6078 [25:54<00:00,  3.91it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [5]:
X = []
y = []
for feature, label in images_data:
  X.append(feature)
  y.append(label)

In [6]:
X = np.array(X) / 255

In [7]:
X.reshape(-1, img_size, img_size, 1)
y = np.array(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [9]:
X_train.shape

(15948, 64, 64, 3)

In [10]:
X_test.shape

(5316, 64, 64, 3)

In [11]:
X_train[0]

array([[[0.91764706, 0.91764706, 0.91764706],
        [0.93333333, 0.93333333, 0.93333333],
        [0.9254902 , 0.9254902 , 0.9254902 ],
        ...,
        [0.50196078, 0.50196078, 0.50196078],
        [0.51764706, 0.51764706, 0.51764706],
        [0.49411765, 0.49411765, 0.49411765]],

       [[0.92156863, 0.92156863, 0.92156863],
        [0.91372549, 0.91372549, 0.91372549],
        [0.90588235, 0.90588235, 0.90588235],
        ...,
        [0.56470588, 0.56470588, 0.56470588],
        [0.54117647, 0.54117647, 0.54117647],
        [0.47843137, 0.47843137, 0.47843137]],

       [[0.91764706, 0.91764706, 0.91764706],
        [0.91764706, 0.91764706, 0.91764706],
        [0.91764706, 0.91764706, 0.91764706],
        ...,
        [0.66666667, 0.66666667, 0.66666667],
        [0.64705882, 0.64705882, 0.64705882],
        [0.60392157, 0.60392157, 0.60392157]],

       ...,

       [[0.17254902, 0.17254902, 0.17254902],
        [0.2745098 , 0.2745098 , 0.2745098 ],
        [0.26666667, 0

In [12]:
y_train.shape

(15948,)

In [13]:
y_test.shape

(5316,)

In [14]:
y_train[0]

2

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range = 30,  
        zoom_range = 0.2,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip = True,  
        vertical_flip=False)  


datagen.fit(X_train)

In [16]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="gelu", input_shape=(64,64,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="gelu"))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="gelu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="gelu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(64,activation="gelu"))
model.add(Dense(3, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          1

In [17]:
model.compile(optimizer = "adam" , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [18]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('gdrive/My Drive/emotions_data/happy_neutral_sad.h5', save_best_only=True, monitor='val_loss', mode='min')

In [19]:
history = model.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=40, batch_size=32,
                   verbose=2, callbacks=[early_stop, mcp_save])

Epoch 1/40
499/499 - 111s - loss: 0.9573 - accuracy: 0.5240 - val_loss: 0.8144 - val_accuracy: 0.6294
Epoch 2/40
499/499 - 112s - loss: 0.7759 - accuracy: 0.6418 - val_loss: 0.7340 - val_accuracy: 0.6527
Epoch 3/40
499/499 - 112s - loss: 0.7159 - accuracy: 0.6691 - val_loss: 0.7168 - val_accuracy: 0.6744
Epoch 4/40
499/499 - 111s - loss: 0.6700 - accuracy: 0.6958 - val_loss: 0.6914 - val_accuracy: 0.6872
Epoch 5/40
499/499 - 111s - loss: 0.6349 - accuracy: 0.7166 - val_loss: 0.6710 - val_accuracy: 0.7005
Epoch 6/40
499/499 - 111s - loss: 0.5915 - accuracy: 0.7418 - val_loss: 0.6388 - val_accuracy: 0.7103
Epoch 7/40
499/499 - 111s - loss: 0.5587 - accuracy: 0.7552 - val_loss: 0.6714 - val_accuracy: 0.7028
Epoch 8/40
499/499 - 111s - loss: 0.5276 - accuracy: 0.7721 - val_loss: 0.6444 - val_accuracy: 0.7129
Epoch 9/40
499/499 - 112s - loss: 0.4943 - accuracy: 0.7886 - val_loss: 0.6364 - val_accuracy: 0.7220
Epoch 10/40
499/499 - 111s - loss: 0.4585 - accuracy: 0.8049 - val_loss: 0.6826 - 